# Data cleaning

In [76]:
import pandas as pd
import numpy as np
import random
import json
from tensorflow.keras.models import load_model
# import pycaret

# Replace double quotes with single quotes for moods JSON arrays
f = open('song_data.csv','r+')
text = f.read()
text = text.replace('"["', '"[?').replace('", "', '?,?').replace('"]"', '?]"') # Changed to "[$Happy$,$Sad$]" for easy replacing later
f.seek(0)
f.write(text)
f.close()

# Importing data
df = pd.read_csv('song_data.csv', index_col=0)
df.drop('uuid', axis=1, inplace=True)
df.dropna(inplace=True) # drop rows with nan values
for col in df.columns:
    if col not in ['id','isSkipped']:
        if col == 'moods':
            df[col] = df[col].apply(lambda x:x.replace('?','"')) # Replaces mood values back to the form ["Happy","Sad"] so it can be loaded by json
        df[col] = df[col].apply(json.loads)
df['activity'] = df.apply(lambda _: '', axis=1) # empty activity column
print('Number of samples: ', df.shape[0])
df.head()

Number of samples:  495


,gyroX,gyroY,gyroZ,accelX,accelY,accelZ,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,,,,,,,
1,"[499.30572509765625, 499.53460693359375, 0.198...","[1.861572265625, 2.49481201171875, 1.022338867...","[1.24359130859375, 1.2359619140625, 1.06048583...","[1.1943359375, 1.201171875, 1.1845703125, 1.18...","[1.1455078125, 1.1591796875, 1.1630859375, 1.1...","[3.68359375, 3.654296875, 3.6748046875, 3.6650...","[139.64, 138.36, 139.64, 140.28]","[30.50567626953125, 30.50567626953125, 30.5056...","[71.3134765625, 71.3134765625, 71.3134765625]","[Depressive, Atmospheric]",0,
2,"[498.1765747070313, 0.98419189453125, 1.579284...","[32.27996826171875, 14.7247314453125, 9.864807...","[497.9248046875, 496.368408203125, 494.9645996...","[2.9326171875, 2.9345703125, 2.728515625, 2.64...","[0.8466796875, 0.74609375, 0.865234375, 15.524...","[2.7548828125, 2.8037109375, 2.806640625, 3.34...","[123.24, 123.24, 139.32, 228.64]","[31.69403076171875, 31.69403076171875, 31.6940...","[67.05322265625, 67.05322265625, 67.0532226562...",[Depressive],0,
3,"[15.76995849609375, 10.65826416015625, 6.87408...","[488.36517333984375, 486.5798950195313, 496.92...","[3.86810302734375, 5.0811767578125, 498.947143...","[14.0107421875, 14.3212890625, 14.232421875, 1...","[14.96484375, 15.212890625, 15.275390625, 15.1...","[3.2021484375, 3.3291015625, 3.375, 3.35644531...","[256.08, 307.84000000000003, 315.2, 301.36, 30...","[32.21771240234375, 32.21771240234375, 32.2177...","[65.850830078125, 65.850830078125, 65.85083007...","[Passionate, Depressive]",0,
4,"[499.93896484375, 499.45068359375, 499.7482299...","[1.82342529296875, 2.74658203125, 1.8844604492...","[1.57928466796875, 1.434326171875, 1.365661621...","[1.7353515625, 1.708984375, 1.7333984375, 1.71...","[13.7841796875, 13.80078125, 13.7744140625, 13...","[2.8232421875, 2.8369140625, 2.8154296875, 2.8...","[127.08, 126.76, 125.48, 124.52]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...",[Elegant],1,
5,"[499.9465942382813, 0.03814697265625, 499.7482...","[2.01416015625, 1.77001953125, 1.7852783203125...","[1.1444091796875, 1.2359619140625, 1.129150390...","[0.4580078125, 0.4609375, 0.453125, 0.43847656...","[13.181640625, 13.1689453125, 13.1787109375, 1...","[2.6806640625, 2.693359375, 2.6875, 2.68652343...","[145.76, 144.48, 146.4, 144.8]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...","[Passionate, Depressive]",1,


In [77]:
# Filtering defective data

defective_ids = []
for idx,row in df.iterrows():
    # defective if temp array only has -40 values
    # defective if any humidity values are above 99.99
    # defective if any optical values are 0
    if len([k for k in row['temp'] if k==-40]) == len(row['temp']) or \
    len([k for k in row['humidity'] if k>99.99]) == len(row['humidity']) or \
    len([k for k in row['optical'] if not k]) == len(row['optical']):
        defective_ids.append(idx)
    
    # if only some values are defective, keep the row, but remove defective values
    # remove -40 temp values, >99.99 humidity values and 0 optical values
    elif (-40 in row['temp']) or len([k for k in row['humidity'] if k>99.99]) or (0 in row['optical']):
        df.at[idx,'temp'] = [k for k in row['temp'] if k!=-40]
        df.at[idx,'humidity'] = [k for k in row['humidity'] if k<99.99]
        df.at[idx,'optical'] = [k for k in row['optical'] if k]
        
    # some gyro/accel data have 40 samples
    # take the last 30 samples for these rows
    for col in df.columns[:6]:
        if len(row[col]) > 30:
            df.at[idx,col] = row[col][-30:]

filtered_df = df[~df.index.isin(defective_ids)].copy() # .copy() to avoid warning
print('%d defective rows: ' % len(defective_ids),defective_ids)

25 defective rows:  [21, 22, 33, 214, 229, 230, 236, 238, 245, 246, 247, 248, 249, 250, 251, 252, 386, 387, 388, 389, 390, 391, 392, 393, 394]


In [78]:
# Add activity from motion recognition model
motion_model_path = 'model2_stackedLSTM.hd5'
motion_model = load_model(motion_model_path)

x = [list(k) for k in filtered_df.iloc[:,:6].values]
x = np.array(x) # (num_samples, 6, 30)
x = np.array([k.T for k in x]) # reshape as (num_samples, 30, 6)

pred = motion_model.predict(x)

encoder_categories = np.array(['Running', 'Walking', 'Working']) # hardcoded categories from 'Physical Activity Classification.ipynb'
filtered_df['activity'] = encoder_categories[np.argmax(pred, axis=1)]
filtered_df = filtered_df.iloc[:,6:].copy() # drop gyro and accel columns

filtered_df.sample(5)

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
370,"[27770.88, 30341.12, 28508.16, 34324.48, 27842...","[31.0394287109375, 31.0394287109375, 31.039428...","[66.339111328125, 66.339111328125, 66.33911132...",[Athletic],0,Walking
282,"[26.28, 26.2, 26.36, 26.84]","[29.2669677734375, 29.2669677734375, 29.266967...","[43.44482421875, 43.44482421875, 43.44482421875]","[Aggressive, Passionate]",1,Working
244,"[1.6, 2, 3.12, 1.04]","[32.550048828125, 32.550048828125, 32.55004882...","[67.742919921875, 67.742919921875, 67.74291992...",[Depressive],1,Walking
314,"[228.8, 226.24, 225.6, 233.68, 234]","[32.6104736328125, 32.6104736328125, 32.610473...","[80.01708984375, 80.01708984375, 80.01708984375]",[Warm],0,Working
315,"[204.4, 186.08, 222.72, 185.68]","[32.6910400390625, 32.68096923828125, 32.68096...","[80.11474609375, 80.01708984375, 80.0170898437...",[Atmospheric],0,Working


In [79]:
# Obtain mean optical, temp and humidity values

for col in filtered_df.columns:
    if col not in ['moods','isSkipped','activity']:
        filtered_df[col] = filtered_df[col].apply(np.mean)
filtered_df.head()

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
1,139.480,30.505676,71.313477,"[Depressive, Atmospheric]",0,Working
2,153.610,31.694031,67.053223,[Depressive],0,Working
3,297.792,32.217712,65.850830,"[Passionate, Depressive]",0,Working
4,125.960,32.429199,64.672852,[Elegant],1,Working
5,145.360,32.429199,64.672852,"[Passionate, Depressive]",1,Working


In [80]:
# One-hot encoding for moods

moods = []
for k in filtered_df['moods'].values:
    moods += list(k)
moods = np.unique(np.array(moods))
for mood in moods:
    mood_values = filtered_df['moods'].astype(str).str.contains(mood)
    filtered_df[mood] = mood_values
filtered_df.drop('moods', axis=1, inplace=True)
print('Added one-hot encoded columns for moods:')
filtered_df.head()

Added one-hot encoded columns for moods:


,optical,temp,humidity,isSkipped,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,,
1,139.480,30.505676,71.313477,0,Working,False,False,True,False,True,False,False,False
2,153.610,31.694031,67.053223,0,Working,False,False,False,False,True,False,False,False
3,297.792,32.217712,65.850830,0,Working,False,False,False,False,True,False,True,False
4,125.960,32.429199,64.672852,1,Working,False,False,False,False,False,True,False,False
5,145.360,32.429199,64.672852,1,Working,False,False,False,False,True,False,True,False


In [81]:
# Invert mood boolean values based on "isSkipped"

for mood in moods:
    filtered_df[mood] = np.abs(filtered_df[mood] - filtered_df['isSkipped'])
filtered_df.drop('isSkipped', axis=1, inplace=True)
print('Invert mood values based on "isSkipped" boolean:')
filtered_df.head()

Invert mood values based on "isSkipped" boolean:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
1,139.480,30.505676,71.313477,Working,0,0,1,0,1,0,0,0
2,153.610,31.694031,67.053223,Working,0,0,0,0,1,0,0,0
3,297.792,32.217712,65.850830,Working,0,0,0,0,1,0,1,0
4,125.960,32.429199,64.672852,Working,1,1,1,1,1,0,1,1
5,145.360,32.429199,64.672852,Working,1,1,1,1,0,1,0,1


# Split into train/test datasets

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# label encoding for activity
le = LabelEncoder()
filtered_df['activity'] = le.fit_transform(filtered_df['activity'].values)
# display(filtered_df.head())

# split into training & testing
x = filtered_df.iloc[:,:4]
y = filtered_df.iloc[:,4:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

print('Training (x, y): ', x_train.shape, y_train.shape)
print('Testing (x, y): ', x_test.shape, y_test.shape)

# create dfs for training and test data for easy prediction later
train_df = x_train.join(y_train)
test_df = x_test.join(y_test)

Training (x, y):  (376, 4) (376, 8)
Testing (x, y):  (94, 4) (94, 8)


# SVM

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm

# model training
# since predict_proba only works with two classes at a time,
# eight svm models are trained for each mood and stored in the dictionary "svm_moods"
svms = {} # key:mood, value:svm model trained on that mood
params = {'C':[10e-3, 10e-2, 0.1, 1,10], 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'decision_function_shape':('ovr', 'ovo')}
for mood in y_train.columns:
    svm_pipe = Pipeline([('scaler', MinMaxScaler()), 
                       ('svm', GridSearchCV(svm.SVC(max_iter=100000, probability=True), params)), ])
    svm_pipe.fit(x_train, y_train.loc[:,mood].values)
    svms[mood] = svm_pipe

C:\Users\rage8\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [86]:
# Define functions for prediction and evaluation of SVM model

from sklearn.metrics import mean_squared_error

# Predict confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
def svm_predict(svms, input_data):
    pred_df = input_data.copy()
    for mood,svm in svms.items():
        prob = svm.predict_proba(pred_df.iloc[:,:-8].values)
        pred_df[mood] = prob[:,1]
    return pred_df
    
# Get loss (MSE) of predicted confidence scores
# Input: DataFrames of actual and predicted moods
def evaluate(df_actual, df_pred):
    df_moods_actual = df_actual.iloc[:,-8:]
    df_moods_pred = df_pred.iloc[:,-8:]
    mse = 0
    for mood in df_moods_actual.columns:
        mse += mean_squared_error(df_moods_actual[mood].values, df_moods_pred[mood].values)
    return mse

In [89]:
train_pred_svm_df = svm_predict(svms, train_df)
test_pred_svm_df = svm_predict(svms, test_df)

print('Test data mood labels (Actual):')
test_df_copy = test_df.copy()
test_df_copy['activity'] = le.inverse_transform(test_df_copy['activity'].values) # convert activity values back to strings
display(test_df_copy.head())
print('Test data mood labels (Predicted):')
test_pred_svm_df['activity'] = le.inverse_transform(test_pred_svm_df['activity'].values)
display(test_pred_svm_df.head())

print('MSE (train): ', evaluate(train_df, train_pred_svm_df))
print('MSE (test): ', evaluate(test_df, test_pred_svm_df))

print('\nPredicted moods for different activities:')
for activity in le.classes_:
    display(test_pred_svm_df[test_pred_svm_df['activity']==activity].head())

Test data mood labels (Actual):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.76,32.610474,80.212402,Walking,0,0,0,0,0,0,1,1
271,2.37,29.266968,43.444824,Working,0,0,0,0,1,0,0,0
24,60.68,32.630615,61.993408,Working,1,1,1,0,1,1,1,1
508,316.40,31.099854,79.947917,Working,1,1,0,1,0,1,1,1
279,21.63,29.266968,43.444824,Working,0,0,1,0,0,1,0,0


Test data mood labels (Predicted):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.76,32.610474,80.212402,Walking,0.444477,0.461243,0.433918,0.614434,0.565558,0.478759,0.466812,0.488496
271,2.37,29.266968,43.444824,Working,0.368976,0.408943,0.433814,0.392726,0.393292,0.443062,0.440716,0.443593
24,60.68,32.630615,61.993408,Working,0.370877,0.433834,0.433496,0.401903,0.406521,0.442337,0.441004,0.444671
508,316.40,31.099854,79.947917,Working,0.377756,0.572731,0.433072,0.609129,0.585900,0.449533,0.441431,0.444646
279,21.63,29.266968,43.444824,Working,0.368852,0.408877,0.433814,0.392551,0.393263,0.443005,0.440697,0.443576


MSE (train):  1.9184582720494436
MSE (test):  2.010537455731105

Predicted moods for different activities:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
239,2.6425,32.590332,68.231201,Running,0.548970,0.552582,0.435147,0.605862,0.424626,0.447465,0.5,0.514262
192,3387.8400,33.486633,82.092285,Running,0.517797,0.406382,0.434798,0.600425,0.493650,0.443203,0.5,0.436829
180,2789.2800,33.255005,80.975342,Running,0.533614,0.429565,0.434829,0.597206,0.489090,0.444480,0.5,0.448770
175,620.5200,33.385925,74.310303,Running,0.539211,0.500000,0.435022,0.606277,0.446767,0.442297,0.5,0.488995
351,1120.3200,32.288208,65.753174,Running,0.539886,0.562766,0.435168,0.588265,0.415792,0.449812,0.5,0.517903


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.760,32.610474,80.212402,Walking,0.444477,0.461243,0.433918,0.614434,0.565558,0.478759,0.466812,0.488496
200,5048.640,35.541077,65.740967,Walking,0.393633,0.416860,0.434267,0.565004,0.418638,0.445846,0.464807,0.491991
152,2232.480,31.321411,81.719971,Walking,0.508338,0.550267,0.433888,0.631398,0.602203,0.488924,0.468436,0.507669
156,376.760,34.030457,72.552490,Walking,0.490041,0.468852,0.434188,0.646347,0.484852,0.465029,0.469796,0.517429
382,2611.712,38.139343,55.944824,Walking,0.370253,0.413558,0.434591,0.398369,0.392828,0.442942,0.466732,0.523384


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
271,2.37,29.266968,43.444824,Working,0.368976,0.408943,0.433814,0.392726,0.393292,0.443062,0.440716,0.443593
24,60.68,32.630615,61.993408,Working,0.370877,0.433834,0.433496,0.401903,0.406521,0.442337,0.441004,0.444671
508,316.40,31.099854,79.947917,Working,0.377756,0.572731,0.433072,0.609129,0.585900,0.449533,0.441431,0.444646
279,21.63,29.266968,43.444824,Working,0.368852,0.408877,0.433814,0.392551,0.393263,0.443005,0.440697,0.443576
86,74.42,34.131165,75.988770,Working,0.357498,0.359160,0.433233,0.391151,0.511830,0.457593,0.441325,0.443202


# Random Forest

In [90]:
from sklearn.ensemble import RandomForestClassifier

params = {'criterion':('gini', 'entropy'), 'max_depth':[k for k in range(1,21)]}
rf = GridSearchCV(RandomForestClassifier(random_state=0), params)
rf.fit(x_train, y_train)
rf.best_params_

{'criterion': 'gini', 'max_depth': 20}

In [91]:
# Predict confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
def rf_predict(rf, input_data):
    pred_df = input_data.copy()
    data = input_data.iloc[:,:-8]
    target = input_data.iloc[:,-8:]
    prob = np.array(rf.predict_proba(data.values))
    for i in range(len(target.columns)):
        mood = target.columns[i]
        pred_df[mood] = prob[i,:,1] # predict_proba returns shape (n_features, n_samples, probs)
    return pred_df

In [94]:
train_pred_rf_df = rf_predict(rf, train_df)
test_pred_rf_df = rf_predict(rf, test_df)

print('Test data mood labels (Actual):')
test_df_copy = test_df.copy()
test_df_copy['activity'] = le.inverse_transform(test_df_copy['activity'].values) # convert activity values back to strings
display(test_df_copy.head())
print('Test data mood labels (Predicted):')
test_pred_rf_df['activity'] = le.inverse_transform(test_pred_rf_df['activity'].values)
display(test_pred_rf_df.head())

print('MSE (train): ', evaluate(train_df, train_pred_rf_df))
print('MSE (test): ', evaluate(test_df, test_pred_rf_df))

print('\nPredicted moods for different activities:')
for activity in le.classes_:
    display(test_pred_rf_df[test_pred_rf_df['activity']==activity].head())

Test data mood labels (Actual):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.76,32.610474,80.212402,Walking,0,0,0,0,0,0,1,1
271,2.37,29.266968,43.444824,Working,0,0,0,0,1,0,0,0
24,60.68,32.630615,61.993408,Working,1,1,1,0,1,1,1,1
508,316.40,31.099854,79.947917,Working,1,1,0,1,0,1,1,1
279,21.63,29.266968,43.444824,Working,0,0,1,0,0,1,0,0


Test data mood labels (Predicted):


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.76,32.610474,80.212402,Walking,0.44,0.30,0.36,0.49,0.35,0.56,0.30,0.43
271,2.37,29.266968,43.444824,Working,0.58,0.59,0.59,0.01,0.93,0.61,0.58,0.60
24,60.68,32.630615,61.993408,Working,0.01,0.92,0.92,0.94,0.76,0.93,0.77,0.95
508,316.40,31.099854,79.947917,Working,0.29,0.59,0.38,0.46,0.46,0.57,0.49,0.28
279,21.63,29.266968,43.444824,Working,0.00,0.02,0.28,0.01,0.00,0.00,0.00,0.69


MSE (train):  0.34989125199685867
MSE (test):  2.5449770708569877

Predicted moods for different activities:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
239,2.6425,32.590332,68.231201,Running,0.83,0.86,0.24,0.79,0.63,0.23,0.78,0.78
192,3387.8400,33.486633,82.092285,Running,0.69,0.06,0.16,0.97,0.16,0.14,0.26,0.14
180,2789.2800,33.255005,80.975342,Running,0.56,0.31,0.34,0.56,0.34,0.10,0.48,0.13
175,620.5200,33.385925,74.310303,Running,0.92,0.77,0.72,0.76,0.76,0.11,0.73,0.76
351,1120.3200,32.288208,65.753174,Running,0.78,0.76,0.34,0.70,0.63,0.67,0.54,0.71


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
353,3717.760,32.610474,80.212402,Walking,0.44,0.30,0.36,0.49,0.35,0.56,0.30,0.43
200,5048.640,35.541077,65.740967,Walking,0.75,0.64,0.17,0.94,0.23,0.75,0.75,0.71
152,2232.480,31.321411,81.719971,Walking,0.86,0.04,0.88,0.92,0.88,0.95,0.88,0.88
156,376.760,34.030457,72.552490,Walking,0.48,0.61,0.60,0.67,0.64,0.60,0.67,0.66
382,2611.712,38.139343,55.944824,Walking,0.80,0.59,0.02,0.59,0.15,0.59,0.69,0.64


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,
271,2.37,29.266968,43.444824,Working,0.58,0.59,0.59,0.01,0.93,0.61,0.58,0.60
24,60.68,32.630615,61.993408,Working,0.01,0.92,0.92,0.94,0.76,0.93,0.77,0.95
508,316.40,31.099854,79.947917,Working,0.29,0.59,0.38,0.46,0.46,0.57,0.49,0.28
279,21.63,29.266968,43.444824,Working,0.00,0.02,0.28,0.01,0.00,0.00,0.00,0.69
86,74.42,34.131165,75.988770,Working,0.22,0.24,0.51,0.15,0.24,0.21,0.53,0.32
